# Proposed Ensemble Models

Given the constraints and objectives, I will consider the following models for the ensemble:
	
    1.	Model 1: LSTM Network on Raw GPS Data
    
>•	Input Data: Sequences of raw GPS data (speed, progress, stride_frequency, etc.).

>•	Architecture: An LSTM network designed to capture temporal dependencies and patterns in the sequential data.

>•	Advantage: LSTMs are well-suited for time-series data and can learn complex temporal dynamics without the need for hand-engineered features like acceleration.

    2.	Model 2: 1D Convolutional Neural Network (1D-CNN)
	
>•	Input Data: The same raw GPS sequences as in Model 1.

>•	Architecture: A 1D-CNN that applies convolutional filters across the time dimension to detect local patterns.

>•	Advantage: CNNs can capture spatial hierarchies and are effective in recognizing patterns in sequences, potentially identifying features like sudden changes in speed or stride frequency.

    3.	Model 3: Transformer-based Model
	
>•	Input Data: Raw GPS sequences and possibly sectionals data.

>•	Architecture: A Transformer model that uses self-attention mechanisms to weigh the importance of different parts of the sequence.

>•	Advantage: Transformers can model long-range dependencies and focus on the most relevant parts of the sequence for prediction.

## Additional Models (Optional):

    4.	Model 4: Gated Recurrent Unit (GRU) Network

>•	Similar to LSTMs but with a simpler architecture, GRUs can be more efficient and may perform better on certain datasets.

>•	Model 5: Temporal Convolutional Network (TCN)

>•	TCNs are designed for sequential data and can capture long-term dependencies using causal convolutions and residual connections.


## Load Parquet Train, Test, and Validaion (VAL) Data:

/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/train_sequences.parquet

/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/test_sequences.parquet

/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/val_sequences.parquet

# Set Environment
import os
import pyspark.sql.functions as F
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, size, when, count
from src.data_preprocessing.data_prep1.data_utils import (
    save_parquet, gather_statistics, initialize_environment,
    load_config, initialize_logging, initialize_spark, 
    identify_and_impute_outliers, identify_and_remove_outliers, process_merged_results_sectionals,
    identify_missing_and_outliers
)

try:
    spark, jdbc_url, jdbc_properties, queries, parquet_dir, log_file = initialize_environment()
    # input("Press Enter to continue...")
except Exception as e:
    print(f"An error occurred during initialization: {e}")
    logging.error(f"An error occurred during initialization: {e}")

In [2]:
# Set Environment
import os
import pyspark.sql.functions as F
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, size, when, count
from src.data_preprocessing.data_prep1.data_utils import (
    save_parquet, gather_statistics, initialize_environment,
    load_config, initialize_logging, initialize_spark, 
    identify_and_impute_outliers, identify_and_remove_outliers, process_merged_results_sectionals,
    identify_missing_and_outliers
)

try:
    spark, jdbc_url, jdbc_properties, queries, parquet_dir, log_file = initialize_environment()
    # input("Press Enter to continue...")
except Exception as e:
    print(f"An error occurred during initialization: {e}")
    logging.error(f"An error occurred during initialization: {e}")

2024-12-30 15:38:29,698 - INFO - Environment setup initialized.


Spark session created successfully.


In [3]:
train_sequences_path = os.path.join(parquet_dir, "train_sequences.parquet")
val_sequences_path = os.path.join(parquet_dir, "val_sequences.parquet")
test_sequences_path = os.path.join(parquet_dir, "test_sequences.parquet")
train_sequences = spark.read.parquet(train_sequences_path)
val_sequences = spark.read.parquet(val_sequences_path)
test_sequences = spark.read.parquet(test_sequences_path)

In [4]:
train_sequences.printSchema()

root
 |-- race_date: date (nullable = true)
 |-- race_number: integer (nullable = true)
 |-- horse_id: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- gate_index: integer (nullable = true)
 |-- course_cd_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- equip_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- surface_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- trk_cond_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- weather_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- med_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- stk_clm_md_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- turf_mud_mark_ohe: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- race_type_ohe: array (nullable = true)
 |    |-- element: d

In [5]:
# Convert to Pandas DataFrame
train_sequences_pd = train_sequences.toPandas()
val_sequences_pd = val_sequences.toPandas()
test_sequences_pd = test_sequences.toPandas()

horse_ids_train = train_sequences_pd["horse_id"].values  # Extract horse_id for training
horse_ids_val = val_sequences_pd["horse_id"].values  # Extract horse_id for validation
horse_ids_test = test_sequences_pd["horse_id"].values  # Extract horse_id for testing

In [6]:
print(train_sequences.select(F.size("past_races_sequence")).distinct().show())

+-------------------------+
|size(past_races_sequence)|
+-------------------------+
|                       10|
+-------------------------+

None


In [7]:
label_distribution = train_sequences.groupBy("label").count().collect()
print(label_distribution)

[Row(label=1, count=5840), Row(label=6, count=3640), Row(label=3, count=5955), Row(label=5, count=4928), Row(label=4, count=5738), Row(label=7, count=5065), Row(label=2, count=5984), Row(label=0, count=5662)]


In [8]:
train_sequences_pd["past_races_sequence"].head()

0    [([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
1    [([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
2    [([0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
3    [([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0...
4    [([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0...
Name: past_races_sequence, dtype: object

In [9]:
import numpy as np

def flatten_sequence(sequence):
    """
    Flattens a sequence of race data into a single NumPy array.
    Ensures uniform array shapes for each time step in the sequence.
    """
    ohe_length = 43 + 26 + 10 + 2 + 8 + 7 + 5 + 4 + 4 + 14  # Total OHE length
    aggregator_length = len(aggregator_cols)  # Length of aggregator columns

    flattened_sequence = []
    for step in sequence:
        # Extract `ohe_flat` or default to zero array
        ohe_flat = step["ohe_flat"] if "ohe_flat" in step else [0.0] * ohe_length

        # Ensure `ohe_flat` has the correct length
        if len(ohe_flat) != ohe_length:
            ohe_flat = [0.0] * ohe_length

        # Extract aggregator values or default to -999.0
        aggregator_values = [step[agg] if agg in step else -999.0 for agg in aggregator_cols]

        # Concatenate `ohe_flat` and aggregator values
        flattened_step = np.array(ohe_flat + aggregator_values)

        # Verify the length of the flattened step
        if len(flattened_step) != (ohe_length + aggregator_length):
            raise ValueError(f"Flattened step has inconsistent length: {len(flattened_step)}")

        flattened_sequence.append(flattened_step)

    return np.array(flattened_sequence)

aggregator_cols = [
    "avg_speed_agg", "max_speed_agg", "final_speed_agg", "avg_accel_agg", 
    "fatigue_agg", "sectional_time_agg", "running_time_agg", "distance_back_agg", 
    "distance_ran_agg", "strides_agg", "max_speed_overall", "min_speed_overall"
]

X_train = np.array([flatten_sequence(seq) for seq in train_sequences_pd["past_races_sequence"]])
y_train = train_sequences_pd["label"].values

In [10]:
X_train = np.array([flatten_sequence(seq) for seq in train_sequences_pd["past_races_sequence"]])
y_train = train_sequences_pd["label"].values

X_val = np.array([flatten_sequence(seq) for seq in val_sequences_pd["past_races_sequence"]])
y_val = val_sequences_pd["label"].values

X_test = np.array([flatten_sequence(seq) for seq in test_sequences_pd["past_races_sequence"]])
y_test = test_sequences_pd["label"].values


In [11]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (42812, 10, 135)
y_train shape: (42812,)


In [12]:
 # Label targets
print(np.unique(y_train))

[0 1 2 3 4 5 6 7]


In [13]:
label_dist = train_sequences.groupBy("label").count().orderBy("label")
label_dist.show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 5662|
|    1| 5840|
|    2| 5984|
|    3| 5955|
|    4| 5738|
|    5| 4928|
|    6| 3640|
|    7| 5065|
+-----+-----+



In [14]:
label_dist = test_sequences.groupBy("label").count().orderBy("label")
label_dist.show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 1055|
|    1| 1115|
|    2| 1130|
|    3| 1107|
|    4| 1073|
|    5|  943|
|    6|  821|
|    7| 1224|
+-----+-----+



In [15]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, horse_ids_train shape: {horse_ids_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}, horse_ids_val shape: {horse_ids_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}, horse_ids_test shape: {horse_ids_test.shape}")

X_train shape: (42812, 10, 135), y_train shape: (42812,), horse_ids_train shape: (42812,)
X_val shape: (11253, 10, 135), y_val shape: (11253,), horse_ids_val shape: (11253,)
X_test shape: (8468, 10, 135), y_test shape: (8468,), horse_ids_test shape: (8468,)


In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, Dropout, GlobalMaxPooling1D, 
                                     Dense, Embedding, Flatten, Concatenate)

# Define the input shapes
time_steps = X_train.shape[1]
features = X_train.shape[2]
num_horses = len(np.unique(horse_ids_train))

# 1) Input layers
input_features = Input(shape=(time_steps, features), name='input_features')
input_horse_id = Input(shape=(1,), name='input_horse_id')

# 2) Horse embedding
embedding = Embedding(input_dim=num_horses, output_dim=32)(input_horse_id)
embedding = Flatten()(embedding)

# 3) CNN layers
x = Conv1D(filters=64, kernel_size=3, activation='relu')(input_features)
x = Dropout(0.2)(x)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)  # or GlobalAveragePooling1D()
x = Dropout(0.2)(x)

# 4) Concatenate CNN output with embedding
concat = Concatenate()([x, embedding])

# 5) Dense layers
dense_out = Dense(64, activation='relu')(concat)
dense_out = Dropout(0.2)(dense_out)

# 6) Output layer (assume 5-class classification example)
output = Dense(5, activation='softmax')(dense_out)

model_cnn = Model(inputs=[input_features, input_horse_id], outputs=output)
model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_cnn.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10, 135)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 8, 64)     │     25,984 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 8, 64)     │          0 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 6, 64)     │     12,352 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_horse_id      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d_5[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 32)     │    339,072 │ input_horse_id[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 64)        │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 32)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 96)        │          0 │ dropout_7[0][0],  │
│ (Concatenate)       │                   │            │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      6,208 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 5)         │        325 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 383,941 (1.46 MB)

 Trainable params: 383,941 (1.46 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True
)

In [20]:
unique, counts = np.unique(y_train, return_counts=True)
train_counts = dict(zip(unique, counts))

print(train_counts)
print(unique)
print(counts)

{0: 5662, 1: 5840, 2: 5984, 3: 5955, 4: 5738, 5: 4928, 6: 3640, 7: 5065}
[0 1 2 3 4 5 6 7]
[5662 5840 5984 5955 5738 4928 3640 5065]


In [21]:
import numpy as np

# Train label counts from your distribution
train_counts = np.array([580, 621, 604, 594, 1889])
total = train_counts.sum()  # 18164
n_classes = len(train_counts)  # 5

class_weight = {}
for i, count_i in enumerate(train_counts):
    class_weight[i] = float(total) / (n_classes * count_i)

print(class_weight)
# Example output:
# {0: 1.463870..., 1: 1.416..., 2: 1.426..., 3: 1.446..., 4: 0.45...}

{0: 1.4786206896551723, 1: 1.3809983896940419, 2: 1.4198675496688742, 3: 1.4437710437710438, 4: 0.453996823716252}


In [ ]:
# Then fit your model:
history = model_cnn.fit(
    [X_train, horse_ids_train],
    y_train,
    epochs=50,
    batch_size=64,
    validation_data=([X_val, horse_ids_val], y_val),
    callbacks=[lr_scheduler, early_stopping, model_checkpoint],
    class_weight=class_weight,
    verbose=1
)




Epoch 1/50


I0000 00:00:1735595250.889758 3092494 service.cc:146] XLA service 0x7f7998001ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735595250.889781 3092494 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
I0000 00:00:1735595250.889783 3092494 service.cc:154]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-12-30 15:47:30.906599: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-30 15:47:30.980392: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300


169/669 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - accuracy: 0.1249 - loss: nan

I0000 00:00:1735595252.372365 3092494 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


669/669 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.1315 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 0.0010
Epoch 2/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1365 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 0.0010
Epoch 3/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1347 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 0.0010
Epoch 4/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1332 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 0.0010
Epoch 5/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1360 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 0.0010
Epoch 6/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1305 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learning_rate: 5.0000e-04
Epoch 7/50
669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1322 - loss: nan - val_accuracy: 0.1277 - val_loss: nan - learnin

In [24]:
# Evaluate:
val_loss, val_accuracy = model_cnn.evaluate([X_val, horse_ids_val], y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1295 - loss: nan
Validation Loss: nan, Validation Accuracy: 0.13089843094348907


In [25]:
preds = model_gru.predict([X_test, horse_ids_test])
print(np.argmax(preds, axis=-1)[:50])  # First 50 predictio

265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# Save the trained model
#model.save('/path/to/save/model.h5')

In [26]:
model_gru.get_layer("embedding_1").get_weights()

[array([[-0.03624932,  0.01595378, -0.01132381, ...,  0.01218581,
          0.03504742,  0.02878574],
        [-0.04518082, -0.04061698,  0.01971302, ...,  0.04646026,
          0.02600035, -0.00787012],
        [-0.00503497, -0.00930778, -0.01403008, ..., -0.02993131,
          0.02693386, -0.00233991],
        ...,
        [-0.03675495,  0.00015531,  0.04402621, ...,  0.00095692,
         -0.03980571, -0.02099963],
        [-0.04129555, -0.00743706,  0.00836102, ..., -0.0076436 ,
          0.00274826, -0.03877602],
        [-0.01163303,  0.0312897 , -0.01761264, ..., -0.00316349,
         -0.02661247, -0.01493084]], dtype=float32)]